In [1]:
# %%
# Import libraries
import betfairlightweight
from betfairlightweight import filters
import pandas as pd
import numpy as np
import os
import datetime
import json
import pytz
import time
from IPython.display import display

# Project libraries
import utils
import constants

# Change this certs path to wherever you're storing your certificates
certs_path = "/home/naing/certs/"
# Change these login details to your own
my_username = constants.USERNAME
my_password = constants.PASSWORD
# my_app_key = constants.API_KEY_DEMO #demo
my_app_key = constants.API_KEY_LIVE #live

trading = betfairlightweight.APIClient(username=my_username,
                                       password=my_password,
                                       app_key=my_app_key,
                                       certs=certs_path)

trading.login()

<LoginResource>

In [2]:
def filter_gh_races(gh_racing_id):
    greyhound_racing_filter = filters.market_filter(
        event_type_ids=[gh_racing_id],
        market_countries=['GB'],
        market_type_codes=['FORECAST'],
        market_start_time={
            'to': (datetime.datetime.utcnow() + datetime.timedelta(hours=6)).strftime("%Y-%m-%dT%TZ") #sydtime 5pm to 7am next day for GB gh races
        }
    )
    #  This returns a list
    gb_gh_events = trading.betting.list_events(
        filter=greyhound_racing_filter)

    if debug:
        # Create a DataFrame with all the events by iterating over each event object
        gb_gh_events_df = pd.DataFrame({
            'Event Name': [event_object.event.name for event_object in gb_gh_events],
            'Event ID': [event_object.event.id for event_object in gb_gh_events],
            'Event Venue': [event_object.event.venue for event_object in gb_gh_events],
            'Country Code': [event_object.event.country_code for event_object in gb_gh_events],
            'Time Zone': [event_object.event.time_zone for event_object in gb_gh_events],
            'Open Date': [event_object.event.open_date for event_object in gb_gh_events],
            'Market Count': [event_object.market_count for event_object in gb_gh_events]
        })

        display(gb_gh_events_df)

    return gb_gh_events

def get_upcoming_games(num_games, market_catalogue_filter):
    market_catalogues = trading.betting.list_market_catalogue(
        filter=market_catalogue_filter,
        market_projection= ['MARKET_START_TIME', 'EVENT'],
        max_results=str(num_games),
        sort='FIRST_TO_START'
    )

    ## show the dataframes if set
    if debug:
        market_types_venueOfTheDay_df = pd.DataFrame({
            'Market Name': [market_cat_object.market_name for market_cat_object in market_catalogues],
            'Market ID': [market_cat_object.market_id for market_cat_object in market_catalogues],
            'Market Start Time': [market_cat_object.market_start_time for market_cat_object in market_catalogues],
            'Total Matched': [market_cat_object.total_matched for market_cat_object in market_catalogues],
            'Venue': [market_cat_object.event.venue for market_cat_object in market_catalogues]
            
        })

        display(market_types_venueOfTheDay_df)

    return market_catalogues

def choose_lay_option_bf(price_filter, lay_selection_index):
    # Request market books
    market_books = trading.betting.list_market_book(
        market_ids=[myRaceID],
        price_projection=price_filter
    )

    # Grab the first market book from the returned list as we only requested one market 
    market_book = market_books[0]
    runners_df = utils.process_runner_books(market_book.runners)
    if debug:
        display(runners_df)

    # Extract a list of IDs for the lay options
    lay_options_ids = []
    for obj in market_book.runners:
        lay_options_ids.append(obj.selection_id)
        
    # if debug:
    #     print(lay_options_ids)
    
    lay_selection_id = lay_options_ids[lay_selection_index]
    print("lay_selection_id = " + str(lay_selection_id))
    fav_price = runners_df.loc[runners_df['Best Lay Price'].idxmin(), 'Best Lay Price']

    return lay_selection_id, fav_price


## TODO
debug = True # Print the data frames
completion_flag = False # completion flag
completion_cnt = 0

while(completion_cnt < 10):
    # %%
    #######################################
    # Filter out only greyhoud races
    #######################################
    gb_gh_events = filter_gh_races(constants.GH_RACING_ID)#Greyhound racing ID

    # %%
    #######################################
    # Extract a list of IDs for the forecast markets
    #######################################
    fc_venue_ids = []
    fc_venue_names = []
    for eventObj in gb_gh_events:
        fc_venue_ids.append(eventObj.event.id)
        fc_venue_names.append(eventObj.event.name)

    fc_venue_ids, fc_venue_names

    # %%
    #######################################
    ## Filter out the next 5 upcoming games
    #######################################
    market_catalogue_filter = filters.market_filter(
        event_ids=fc_venue_ids)

    market_catalogues = get_upcoming_games(5, market_catalogue_filter) 

    # %%
    #######################################
    ## Check the start time of the next game
    ## and sleep until 15 seconds before the game
    #######################################
    start_time, time_gap, myRaceID, myRaceVenue = utils.get_next_market(market_catalogues)
    # assert myRaceID == 0, "ERROR: myRaceID = 0" #TODO: remove in production

    if(time_gap > datetime.timedelta(seconds=constants.PREBET_DELAY)):
        time_to_sleep = (time_gap - datetime.timedelta(seconds=constants.PREBET_DELAY)).seconds
        print("Sleeping for " + str(time_to_sleep) + " seconds")
        time.sleep(time_to_sleep) #TODO
        print("Sleeping done")
    else:
        print("Don't need to sleep")

    #######################################
    # Get odds from NEDS
    #######################################
    lay_selection_index = utils.choose_lay_option_neds(myRaceVenue)

    if(lay_selection_index == -1):
        print("ERROR!!! 2 same odds found")
        assert()
        
    elif(lay_selection_index == -2):
        print("ERROR!!! less than 6 runners")
        # assert()

    elif(lay_selection_index == -3):
        print("ERROR!!! NEDS API failed")
        assert()
    else:
        print("Lay option found successfully")
        print(lay_selection_index)

    # %%
    #######################################
    # Get a list of lay options from betfair
    #######################################
    # Create a price filter. Get all traded and offer data
    price_filter = filters.price_projection(
        price_data=['EX_BEST_OFFERS']
    )
    lay_selection_id, fav_price = choose_lay_option_bf(price_filter, lay_selection_index)

    # %%
    #######################################
    # Choose the liability amount
    # random from a list of 10 options
    #######################################
    liability_options = []
    if(len(liability_options) == 0):
        liability_options = [5, 5, 5, 5, 5, 5, 5, 5, 100, 100] #not sure why this can't be moved into constants.py
    print("liability_options [BEFORE] = ", liability_options, ", LENGTH =", len(liability_options))

    [liability_amount] = np.random.choice(liability_options, size=1)
    print("Chosen Liability amount = $", liability_amount)

    liability_options.remove(liability_amount)
    print("liability_options [AFTER] = ", liability_options, ", LENGTH =", len(liability_options))

    # %%
    #######################################
    # Choose order type (limit order or market_on_close order - choose the latter)
    #######################################
    order_filter = filters.limit_order(
        price=str(fav_price),
        persistence_type='LAPSE',
        bet_target_type='PAYOUT',
        bet_target_size=str(liability_amount) 
    )
    print(order_filter)

    #######################################
    # Create a place instruction filter
    #######################################
    instructions_filter = filters.place_instruction(
        selection_id = str(lay_selection_id),
        side="LAY",
        order_type = "LIMIT", # fixed price order
        limit_order=order_filter
    )
    print(instructions_filter)


    # %%
    #######################################
    # Place the order
    #######################################
    order = trading.betting.place_orders(
        market_id = myRaceID,
        customer_strategy_ref='Naing_maker',
        instructions=[instructions_filter]
    )

    # %%
    #######################################
    # Make sure the order is matched fully
    #######################################
    # utils.ensure_order_matched(myRaceID, lay_selection_index, price_filter)
    fullyMatched_flag = False
    while fullyMatched_flag == False:
        ## get the current order
        current_orders = trading.betting.list_current_orders(
            market_ids=[myRaceID])
        single_current_order = current_orders._data['currentOrders'][0]
        print(single_current_order)

        unmatched_size = single_current_order['sizeRemaining']
        print('unmatched size = ', unmatched_size)

        ## if there's unmatched order, replace it with new price, else set the fullyMatched flag
        if float(unmatched_size) != 0:
            print("Order is not fully matched yet")

            ## Get the new best Lay price
            lay_selection_id, fav_price = choose_lay_option_bf(price_filter, lay_selection_index)

            ## Replace the unmatched order with the new price
            replace_instructions_filter = filters.replace_instruction(
                bet_id= single_current_order['betId'],
                new_price= fav_price
            )
            print(replace_instructions_filter)

            newOrder = trading.betting.replace_orders(
                    market_id = myRaceID,
                    customer_ref='Naing_maker_Replaced',
                    instructions=[replace_instructions_filter]
                )

            print("Replaced order")
            time.sleep(5)
        else:
            print("Full order matched")
            fullyMatched_flag = True

    #######################################
    # Check if the last bet has settled and note the result
    # Only if the previous race is settled, start next game 
    #######################################
    settled_flag = False
    while settled_flag == False:
        cleared_orders = trading.betting.list_cleared_orders(
            bet_status="SETTLED",
            market_ids=[myRaceID])

        if len(cleared_orders._data['clearedOrders']) != 0:
            settled_flag = True

            # Create a DataFrame from the orders
            betResult = pd.DataFrame(cleared_orders._data['clearedOrders'])
            print(betResult)

            betOutcome = betResult['betOutcome'][0]
            profit = 0
            for i in betResult['profit']:
                profit = profit + float(i)

            ## Record the results into a csv file
            utils.write_to_file(constants.F_NAME, myRaceID, betOutcome, profit)

            completion_flag = True # we are ready for next game
            completion_cnt = completion_cnt + 1

        else:
            print("Sleep 60 seconds before checking again if market is settled")
            time.sleep(60) # TODO:Check again in 60 seconds
    
    print("Sleeping for some time before starting the next game")
    time.sleep(np.random.randint(60,180))

,Event Name,Event ID,Event Venue,Country Code,Time Zone,Open Date,Market Count
0,Central Park (F/C) 3rd Jul,31567489,Central Park,GB,Europe/London,2022-07-03 17:09:00,5
1,Henlow (F/C) 3rd Jul,31567505,Henlow,GB,Europe/London,2022-07-03 17:17:00,5
2,Doncaster (F/C) 3rd Jul,31567440,Doncaster,GB,Europe/London,2022-07-03 09:46:00,1
3,Kinsley (F/C) 3rd Jul,31567458,Kinsley,GB,Europe/London,2022-07-03 13:04:00,12
4,Pelaw Grange (F/C) 3rd Jul,31567428,Pelaw Grange,GB,Europe/London,2022-07-03 10:43:00,3
5,Towcester (F/C) 3rd Jul,31567467,Towcester,GB,Europe/London,2022-07-03 12:57:00,14
6,Swindon (F/C) 3rd Jul,31567482,Swindon,GB,Europe/London,2022-07-03 17:19:00,5
7,Harlow (F/C) 3rd Jul,31567452,Harlow,GB,Europe/London,2022-07-03 09:53:00,2
8,Sunderland (F/C) 3rd Jul,31567420,Sunderland,GB,Europe/London,2022-07-03 10:06:00,2


,Market Name,Market ID,Market Start Time,Total Matched,Venue
0,Forecast,1.200715893,2022-07-03 12:24:00,675.51108,Harlow
1,Forecast,1.200715820,2022-07-03 12:28:00,0.00000,Pelaw Grange
2,Forecast,1.200715858,2022-07-03 12:33:00,0.00000,Doncaster
3,Forecast,1.200715791,2022-07-03 12:36:00,46.06160,Sunderland
4,Forecast,1.200715894,2022-07-03 12:39:00,0.00000,Harlow


Time Now: 
2022-07-03 12:27:39
Found the market to lay: Name = pelaw-grange id = 1.200715820
market Name =  Forecast
Market Start Time: 2022-07-03 12:28:00+00:00
Sleeping for 0 seconds
Sleeping done
['PELAW GRANGE BAGS', 'GOOD', '2', '3', '4', '5', '6', '7', '8', '9', '10', '435m', 'Race 8', '16s', 'RACE FIELD', 'COMMENTS', '$ TRACKER', 'FLUCTUATIONS', 'WIN / PLACE SAME RACE MULTI BLENDED MBO', 'LIVE VIDEO', 'Pelaw Grange Bags', 'FIXEDSTARTING PRICE', 'RUNNERS']
end of part1
['RUNNERS', 'WIN', 'PLC', '231', '1. Wise Bandit (1)', 'T: G Walker', '4.80', '2.30', '524', '2. Snazzy Jacko (2)', 'T: G Walker', '5.00', '2.35', '21', '3. Broken Brella (3)', 'T: C Watson', '6.50', '2.80', '316', '4. Hurleys Spring (4)', 'T: P N Richardson', '4.20', '2.10', '2121', '5. Liffeyside Mutaz (5)', 'T: J Watson', '3.50', '1.85', '3312', '6. Dalton Opus (6)', 'T: P A Bedding', '5.00', '2.35', 'Event Rules: Only two place dividends paid.', 'ODDS VS EVENS', 'ODDS', '3 RUNNERS', '1.75', 'VS', 'EVENS', '3 RU

,Selection ID,Best Back Price,Best Back Size,Best Lay Price,Best Lay Size,Last Price Traded,Total Matched,Status,Removal Date,Adjustment Factor
0,41419585,22.0,15.94,27.0,5.47,22.0,28.84,ACTIVE,None,None
1,41419586,32.0,35.68,50.0,36.84,30.0,35.37,ACTIVE,None,None
2,41419587,28.0,6.32,34.0,33.60,28.0,7.10,ACTIVE,None,None
3,41419588,25.0,7.73,30.0,40.69,23.0,110.28,ACTIVE,None,None
4,41419589,34.0,8.93,42.0,27.60,38.0,22.25,ACTIVE,None,None
5,41419592,25.0,9.74,32.0,10.75,26.0,8.25,ACTIVE,None,None
6,41419593,36.0,46.26,48.0,12.44,44.0,14.11,ACTIVE,None,None
7,41419594,32.0,7.86,40.0,8.40,34.0,2.73,ACTIVE,None,None
8,41419595,29.0,9.77,36.0,10.60,30.0,9.47,ACTIVE,None,None
9,41419596,40.0,5.97,50.0,26.27,34.0,3.19,ACTIVE,None,None


lay_selection_id = 41419616
liability_options [BEFORE] =  [5, 5, 5, 5, 5, 5, 5, 5, 100, 100] , LENGTH = 10
Chosen Liability amount = $ 5
liability_options [AFTER] =  [5, 5, 5, 5, 5, 5, 5, 100, 100] , LENGTH = 9
{'price': '21.0', 'persistenceType': 'LAPSE', 'betTargetType': 'PAYOUT', 'betTargetSize': '5'}
{'orderType': 'LIMIT', 'selectionId': '41419616', 'side': 'LAY', 'limitOrder': {'price': '21.0', 'persistenceType': 'LAPSE', 'betTargetType': 'PAYOUT', 'betTargetSize': '5'}}
{'betId': '273069534489', 'marketId': '1.200715820', 'selectionId': 41419616, 'handicap': 0.0, 'priceSize': {'price': 21.0, 'size': 0.24}, 'bspLiability': 0.0, 'side': 'LAY', 'status': 'EXECUTABLE', 'persistenceType': 'LAPSE', 'orderType': 'LIMIT', 'placedDate': '2022-07-03T12:27:46.000Z', 'averagePriceMatched': 0.0, 'sizeMatched': 0.0, 'sizeRemaining': 0.24, 'sizeLapsed': 0.0, 'sizeCancelled': 0.0, 'sizeVoided': 0.0, 'regulatorCode': 'MALTA LOTTERIES AND GAMBLING AUTHORITY', 'customerStrategyRef': 'Naing_maker'}


...

,Event Name,Event ID,Event Venue,Country Code,Time Zone,Open Date,Market Count
0,Central Park (F/C) 3rd Jul,31567489,Central Park,GB,Europe/London,2022-07-03 17:09:00,5
1,Henlow (F/C) 3rd Jul,31567505,Henlow,GB,Europe/London,2022-07-03 17:17:00,5
2,Doncaster (F/C) 3rd Jul,31567440,Doncaster,GB,Europe/London,2022-07-03 09:46:00,1
3,Kinsley (F/C) 3rd Jul,31567458,Kinsley,GB,Europe/London,2022-07-03 13:04:00,12
4,Pelaw Grange (F/C) 3rd Jul,31567428,Pelaw Grange,GB,Europe/London,2022-07-03 10:43:00,3
5,Towcester (F/C) 3rd Jul,31567467,Towcester,GB,Europe/London,2022-07-03 12:57:00,14
6,Swindon (F/C) 3rd Jul,31567482,Swindon,GB,Europe/London,2022-07-03 17:19:00,5
7,Sunderland (F/C) 3rd Jul,31567420,Sunderland,GB,Europe/London,2022-07-03 10:06:00,2
8,Harlow (F/C) 3rd Jul,31567452,Harlow,GB,Europe/London,2022-07-03 09:53:00,1


,Market Name,Market ID,Market Start Time,Total Matched,Venue
0,Forecast,1.200715820,2022-07-03 12:28:00,0.0000,Pelaw Grange
1,Forecast,1.200715858,2022-07-03 12:33:00,0.0000,Doncaster
2,Forecast,1.200715791,2022-07-03 12:36:00,46.0616,Sunderland
3,Forecast,1.200715894,2022-07-03 12:39:00,0.0000,Harlow
4,Forecast,1.200715821,2022-07-03 12:44:00,60.2344,Pelaw Grange


Time Now: 
2022-07-03 12:31:17
Found the market to lay: Name = doncaster id = 1.200715858
market Name =  Forecast
Market Start Time: 2022-07-03 12:33:00+00:00
Sleeping for 82 seconds
Sleeping done
['DONCASTER BAGS', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '450m', '450Mtrs (B6)', '16s', 'RACE FIELD', 'COMMENTS', '$ TRACKER', 'FLUCTUATIONS', 'WIN / PLACE SAME RACE MULTI BLENDED MBO', 'LIVE VIDEO', 'Doncaster Bags', 'FIXEDSTARTING PRICE', 'RUNNERS']
end of part1
['RUNNERS', 'WIN', 'PLC', '1343', '1. Blakes Lace (1)', 'T: J Simpson', '6.50', '2.80', '5554', '2. Lees Little Girl (2)', 'T: L J Macmanus', '10.00', '3.80', '2131', '3. Hulla Head (3)', 'T: S Atkinson', '3.60', '1.90', '3634', '4. Swift Radha (4)', 'T: G Vincent', '3.40', '1.80', '6315', '5. Swift Emphasis (5)', 'T: D L Cross', '5.00', '2.30', '4163', '6. Eden Lane (6)', 'T: M Haythorne', '3.70', '1.90', 'Event Rules: Only two place dividends paid.', 'ODDS VS EVENS', 'ODDS', '3 RUNNERS', '1.80', 'VS', 'EVE

,Selection ID,Best Back Price,Best Back Size,Best Lay Price,Best Lay Size,Last Price Traded,Total Matched,Status,Removal Date,Adjustment Factor
0,41419585,48.0,7.64,70.0,12.04,48.0,46.62,ACTIVE,None,None
1,41419586,32.0,5.48,38.0,8.86,32.0,18.40,ACTIVE,None,None
2,41419587,32.0,13.49,40.0,11.40,34.0,38.45,ACTIVE,None,None
3,41419588,44.0,15.55,65.0,9.47,46.0,33.85,ACTIVE,None,None
4,41419589,36.0,14.40,46.0,21.05,40.0,29.23,ACTIVE,None,None
5,41419592,60.0,11.95,120.0,11.32,65.0,26.33,ACTIVE,None,None
6,41419593,80.0,15.26,100.0,10.74,80.0,27.03,ACTIVE,None,None
7,41419594,65.0,32.42,120.0,17.01,70.0,10.62,ACTIVE,None,None
8,41419595,100.0,13.39,180.0,15.05,90.0,21.25,ACTIVE,None,None
9,41419596,85.0,18.67,140.0,15.22,95.0,10.62,ACTIVE,None,None


lay_selection_id = 41419608
liability_options [BEFORE] =  [5, 5, 5, 5, 5, 5, 5, 5, 100, 100] , LENGTH = 10
Chosen Liability amount = $ 5
liability_options [AFTER] =  [5, 5, 5, 5, 5, 5, 5, 100, 100] , LENGTH = 9
{'price': '15.5', 'persistenceType': 'LAPSE', 'betTargetType': 'PAYOUT', 'betTargetSize': '5'}
{'orderType': 'LIMIT', 'selectionId': '41419608', 'side': 'LAY', 'limitOrder': {'price': '15.5', 'persistenceType': 'LAPSE', 'betTargetType': 'PAYOUT', 'betTargetSize': '5'}}
{'betId': '273070046009', 'marketId': '1.200715858', 'selectionId': 41419608, 'handicap': 0.0, 'priceSize': {'price': 15.5, 'size': 0.34}, 'bspLiability': 0.0, 'side': 'LAY', 'status': 'EXECUTION_COMPLETE', 'persistenceType': 'LAPSE', 'orderType': 'LIMIT', 'placedDate': '2022-07-03T12:32:45.000Z', 'matchedDate': '2022-07-03T12:32:45.000Z', 'averagePriceMatched': 14.5, 'sizeMatched': 0.34, 'sizeRemaining': 0.0, 'sizeLapsed': 0.0, 'sizeCancelled': 0.0, 'sizeVoided': 0.0, 'regulatorCode': 'MALTA LOTTERIES AND GAMBLIN